## Setup

In [4]:
import dotenv, os, openai, csv

EXAMPLES_DIR = 'examples'
RESULTS_FILENAME = 'results.csv'

dotenv.load_dotenv()
gpt = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def prompt_gpt(prompt: str) -> str:
    response = gpt.chat.completions.create(
        model='gpt-3.5',
        messages=[
            {
                'role': 'user',
                'content': prompt,
            }
        ]
    )
    output = response.choices[0].message.content.strip()
    return output

def format_examples(examples: list[dict[str, str]]) -> str:
    addition = ""
    
    for example in examples:
        addition += f"Q: {example['q']}"
        addition += "\n"
        addition += f"A: {example['a']}"
        addition += "\n"
    
    return addition

## Load examples

Data is formatted in CSV files as follows:
```
q_1,a_1
q2,a_2
...
q_n,a_n
```

In [5]:
examples_by_task_class = {}  # TODO add examples into CSVs (ideally 12 or 16 total)

for filename in os.listdir(EXAMPLES_DIR):
    if filename.endswith('.csv'):
        path = os.path.join(EXAMPLES_DIR, filename)
        
        with open(path, newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            task_class = filename[:-4]
            examples_by_task_class[task_class] = [{'q': row[0], 'a': row[1]} for row in reader]
            
tasks_by_task_class = {  # TODO for each task class, (key, value) = (task class, answer)
    
}

## Run trials

In [6]:
results = [["Task Class", "Case", "Response"]]

for task_class, examples in examples_by_task_class.items():
    half_n = len(examples) // 2
    # case 1: use n / 2 examples
        # case 1a: use the first n / 2 examples
    prompt = format_examples(examples[:half_n])
    prompt += f"Q: {tasks_by_task_class[task_class]}"
    response = prompt_gpt(prompt)
    results.append([task_class, "1a", response])
        # case 1b: use the other n / 2 examples
    prompt = format_examples(examples[half_n:])
    prompt += f"Q: {tasks_by_task_class[task_class]}"
    response = prompt_gpt(prompt)
    results.append([task_class, "1b", response])
    # case 2: use all n examples
    prompt = format_examples(examples)
    prompt += f"Q: {tasks_by_task_class[task_class]}"
    response = prompt_gpt(prompt)
    results.append([task_class, "2", response])
    # case 3: use n / 2 examples, duplicated (for n total examples)
        # case 3a: use the first n / 2 examples
    prompt = format_examples(examples[:half_n] + examples[:half_n])
    prompt += f"Q: {tasks_by_task_class[task_class]}"
    response = prompt_gpt(prompt)
    results.append([task_class, "1a", response])
        # case 3b: use the other n / 2 examples
    prompt = format_examples(examples[half_n:] + examples[half_n:])
    prompt += f"Q: {tasks_by_task_class[task_class]}"
    response = prompt_gpt(prompt)
    results.append([task_class, "1a", response])
    # TODO other cases

with open(RESULTS_FILENAME, 'w', newline='') as f:
    writer = csv.writer(f)
    
    for row in results:
        writer.writerow(row)